### This notebook will create a table of a the unit's training status from from raw .csv exports.

The goal is to have a semi-automated process that works from .csv files that the unit training manager can download. Then, the analyst simply has to drop the files into the appropriate folder on a weekly (or any timeframe) basis, and run this workbook. The resulting table will be linked to Tableau for visualization.

Note: All names used are fictitious.

### Step 1: Loading our libraries

In [2]:
#pip install xlrd
#pip install findspark
#pip install pyjanitor

In [23]:
import pandas as pd
import janitor
import glob
import numpy as np

### Step 2: Reading the data

For our example, we have data from 6 different .csv files. We will read in all of the .csv files in our folder, and combine them into a single Pandas dataframe.

In [24]:
import os
pwd = os.getcwd()

path = pwd + "/raw_data/"

all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
union = pd.concat(df_from_each_file, ignore_index=True)

union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rank               1705 non-null   object 
 1   First Name         1705 non-null   object 
 2   Last Name          1705 non-null   object 
 3   Course Name        1705 non-null   object 
 4   Enrollment Date    1705 non-null   object 
 5   Grade              463 non-null    float64
 6   Date Completed     1450 non-null   object 
 7   Completion Status  1705 non-null   object 
dtypes: float64(1), object(7)
memory usage: 106.7+ KB


Convert all columns containing dates to the proper "datetime" format:

In [25]:
union['Enrollment Date'] = union['Enrollment Date'].astype('datetime64[ns]')
union['Date Completed'] = union['Date Completed'].astype('datetime64[ns]')

In [26]:
print(union)

      Rank First Name   Last Name  \
0      Maj     ASHLEY  WILLIAMSON   
1      Maj     ASHLEY  WILLIAMSON   
2      Maj     ASHLEY  WILLIAMSON   
3      Maj     ASHLEY  WILLIAMSON   
4      Maj     ASHLEY  WILLIAMSON   
...    ...        ...         ...   
1700  TSgt      LEMAR      HANSON   
1701  TSgt      LEMAR      HANSON   
1702  TSgt      LEMAR      HANSON   
1703  TSgt      LEMAR      HANSON   
1704  TSgt      LEMAR      HANSON   

                                            Course Name Enrollment Date  \
0            *Cyber Awareness Challenge 2021 (ZZ133098)      2021-03-22   
1            *Cyber Awareness Challenge (ZZ133098)-ADLS      2021-04-22   
2                     *Force Protection (ZZ133079)-ADLS      2021-04-24   
3                            SECDEF OPSEC Campaign-ADLS      2021-04-26   
4             Cyber Awareness Challenge 2021 (ZZ133098)      2021-10-06   
...                                                 ...             ...   
1700  Controlled Unclassified 

### Step 3: Preparing and cleaning the data

First, we want to add columns for 1, 2, and 3 year due dates:

In [27]:
import datetime

In [28]:
from datetime import timedelta
from datetime import datetime

In [29]:
union['Due Date 1 yr'] = union['Date Completed']+timedelta(days=365)
union['Due Date 2 yr'] = union['Date Completed']+timedelta(days=365*2)
union['Due Date 3 yr'] = union['Date Completed']+timedelta(days=365*3)

print(union)

      Rank First Name   Last Name  \
0      Maj     ASHLEY  WILLIAMSON   
1      Maj     ASHLEY  WILLIAMSON   
2      Maj     ASHLEY  WILLIAMSON   
3      Maj     ASHLEY  WILLIAMSON   
4      Maj     ASHLEY  WILLIAMSON   
...    ...        ...         ...   
1700  TSgt      LEMAR      HANSON   
1701  TSgt      LEMAR      HANSON   
1702  TSgt      LEMAR      HANSON   
1703  TSgt      LEMAR      HANSON   
1704  TSgt      LEMAR      HANSON   

                                            Course Name Enrollment Date  \
0            *Cyber Awareness Challenge 2021 (ZZ133098)      2021-03-22   
1            *Cyber Awareness Challenge (ZZ133098)-ADLS      2021-04-22   
2                     *Force Protection (ZZ133079)-ADLS      2021-04-24   
3                            SECDEF OPSEC Campaign-ADLS      2021-04-26   
4             Cyber Awareness Challenge 2021 (ZZ133098)      2021-10-06   
...                                                 ...             ...   
1700  Controlled Unclassified 

Combine first and last name columns:

In [30]:
union['Name'] = union[['Last Name', 'First Name']].apply(lambda x: ' '.join(x), axis=1)

We need to add entries for every course, that way if someone hasn't started a course, they will show up as incomplete for our visualization:

In [31]:
union = union.complete('Name', 'Course Name')

In [32]:
union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7981 entries, 0 to 7980
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               7981 non-null   object        
 1   Course Name        7981 non-null   object        
 2   Rank               1705 non-null   object        
 3   First Name         1705 non-null   object        
 4   Last Name          1705 non-null   object        
 5   Enrollment Date    1705 non-null   datetime64[ns]
 6   Grade              463 non-null    float64       
 7   Date Completed     1450 non-null   datetime64[ns]
 8   Completion Status  1705 non-null   object        
 9   Due Date 1 yr      1450 non-null   datetime64[ns]
 10  Due Date 2 yr      1450 non-null   datetime64[ns]
 11  Due Date 3 yr      1450 non-null   datetime64[ns]
dtypes: datetime64[ns](5), float64(1), object(6)
memory usage: 748.3+ KB


Sanity check a few of your values:

In [33]:
n = len(pd.unique(union['Name']))
print("Number of unique names:", n)

Number of unique names: 75


In [34]:
m = len(pd.unique(union['Course Name']))
print("Number of unique courses:", m)

Number of unique courses: 106


Header cleaning:

In [35]:
# Column headers: lower case + remove spaces and the following characters: ,;{}()=  

union.columns = union.columns.str.lower()
union.columns = union.columns.str.replace(' ', '_')

problematic_chars = ',;{}()=?'
for c in problematic_chars:
        union.columns = union.columns.str.replace(c, '')

<ipython-input-35-3b060983acb1>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  union.columns = union.columns.str.replace(c, '')


In [36]:
union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7981 entries, 0 to 7980
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               7981 non-null   object        
 1   course_name        7981 non-null   object        
 2   rank               1705 non-null   object        
 3   first_name         1705 non-null   object        
 4   last_name          1705 non-null   object        
 5   enrollment_date    1705 non-null   datetime64[ns]
 6   grade              463 non-null    float64       
 7   date_completed     1450 non-null   datetime64[ns]
 8   completion_status  1705 non-null   object        
 9   due_date_1_yr      1450 non-null   datetime64[ns]
 10  due_date_2_yr      1450 non-null   datetime64[ns]
 11  due_date_3_yr      1450 non-null   datetime64[ns]
dtypes: datetime64[ns](5), float64(1), object(6)
memory usage: 748.3+ KB


### Step 4: Data Preparation

Here, we want to end up with a table that has each person's course information and due dates. Several of the courses are listed under different version names, so we need to find the latest course that they completed for each "group," and use that for our final table. The end product should be a list of personnel with the last completion date, due date, and number of days overdue for all of the courses we want to track.

In [37]:
today = datetime.now()
print(today)

2022-02-07 18:35:34.152944


Cyber Awareness:

In [38]:
df = union[union['course_name'].str.contains('Cyber Awareness')]

cyber_df = (df.sort_values(['name', 'due_date_1_yr'], ascending=[True, False]).drop_duplicates(['name']).reset_index(drop=True))

cyber_df.drop(["due_date_2_yr", "due_date_3_yr"], axis = 1, inplace = True)
cyber_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,due_date_1_yr
0,ALEXANDER SARAH,*Cyber Awareness Challenge 2021 (ZZ133098),Maj,SARAH,ALEXANDER,2021-03-21,NaN,2021-09-24,Yes,2022-09-24
1,BARKER DAVID,Cyber Awareness Challenge 2021 (ZZ133098),Maj,DAVID,BARKER,2021-09-16,0.0,2022-01-06,Yes,2023-01-06
2,BARKER JESSE,*Cyber Awareness Challenge 2021 (ZZ133098),SSgt,JESSE,BARKER,2021-03-22,NaN,2021-09-24,Yes,2022-09-24
3,BAXTER EMELIE,Cyber Awareness Challenge 2021 (ZZ133098),TSgt,EMELIE,BAXTER,2021-10-06,0.0,2021-11-30,Yes,2022-11-30
4,BOND JIMMY,*Cyber Awareness Challenge 2021 (ZZ133098),Capt,JIMMY,BOND,2021-03-22,NaN,2021-09-24,Yes,2022-09-24
...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,*Cyber Awareness Challenge 2021 (ZZ133098),GS-09,DEREK,WILLIAMS,2021-03-22,NaN,2021-09-24,Yes,2022-09-24
71,WILLIAMS GABRIEL,Cyber Awareness Challenge 2021 (ZZ133098),GS-12,GABRIEL,WILLIAMS,2021-09-16,0.0,2021-12-13,Yes,2022-12-13
72,WILLIAMSON ASHLEY,*Cyber Awareness Challenge 2021 (ZZ133098),Maj,ASHLEY,WILLIAMSON,2021-03-22,NaN,2021-09-24,Yes,2022-09-24
73,WONG JACQUELINE,*Cyber Awareness Challenge 2021 (ZZ133098),GS-13,JACQUELINE,WONG,2021-03-22,NaN,2021-09-24,Yes,2022-09-24


In [39]:
cyber_df['days_overdue'] = today - cyber_df['due_date_1_yr']

cyber_df['days_overdue'] = round(cyber_df['days_overdue'] / pd.Timedelta(1, unit='d'))

cyber_df.loc[cyber_df['days_overdue'] < 0,'days_overdue'] = 0

cyber_df.rename(columns={'due_date_1_yr':'date_due'}, inplace=True)

cyber_df.replace({np.nan: 0}, inplace = True)

cyber_df['completion_status'] = np.where( ( (cyber_df['enrollment_date'] == 0) | (cyber_df['days_overdue'] > 0)), "No", "Yes")

cyber_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,*Cyber Awareness Challenge 2021 (ZZ133098),Maj,SARAH,ALEXANDER,2021-03-21,0.0,2021-09-24,Yes,2022-09-24,0.0
1,BARKER DAVID,Cyber Awareness Challenge 2021 (ZZ133098),Maj,DAVID,BARKER,2021-09-16,0.0,2022-01-06,Yes,2023-01-06,0.0
2,BARKER JESSE,*Cyber Awareness Challenge 2021 (ZZ133098),SSgt,JESSE,BARKER,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
3,BAXTER EMELIE,Cyber Awareness Challenge 2021 (ZZ133098),TSgt,EMELIE,BAXTER,2021-10-06,0.0,2021-11-30,Yes,2022-11-30,0.0
4,BOND JIMMY,*Cyber Awareness Challenge 2021 (ZZ133098),Capt,JIMMY,BOND,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,*Cyber Awareness Challenge 2021 (ZZ133098),GS-09,DEREK,WILLIAMS,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
71,WILLIAMS GABRIEL,Cyber Awareness Challenge 2021 (ZZ133098),GS-12,GABRIEL,WILLIAMS,2021-09-16,0.0,2021-12-13,Yes,2022-12-13,0.0
72,WILLIAMSON ASHLEY,*Cyber Awareness Challenge 2021 (ZZ133098),Maj,ASHLEY,WILLIAMSON,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
73,WONG JACQUELINE,*Cyber Awareness Challenge 2021 (ZZ133098),GS-13,JACQUELINE,WONG,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0


In [40]:
cyber_df = cyber_df.assign(course_name = "Cyber Awareness")
cyber_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Cyber Awareness,Maj,SARAH,ALEXANDER,2021-03-21,0.0,2021-09-24,Yes,2022-09-24,0.0
1,BARKER DAVID,Cyber Awareness,Maj,DAVID,BARKER,2021-09-16,0.0,2022-01-06,Yes,2023-01-06,0.0
2,BARKER JESSE,Cyber Awareness,SSgt,JESSE,BARKER,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
3,BAXTER EMELIE,Cyber Awareness,TSgt,EMELIE,BAXTER,2021-10-06,0.0,2021-11-30,Yes,2022-11-30,0.0
4,BOND JIMMY,Cyber Awareness,Capt,JIMMY,BOND,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Cyber Awareness,GS-09,DEREK,WILLIAMS,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
71,WILLIAMS GABRIEL,Cyber Awareness,GS-12,GABRIEL,WILLIAMS,2021-09-16,0.0,2021-12-13,Yes,2022-12-13,0.0
72,WILLIAMSON ASHLEY,Cyber Awareness,Maj,ASHLEY,WILLIAMSON,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0
73,WONG JACQUELINE,Cyber Awareness,GS-13,JACQUELINE,WONG,2021-03-22,0.0,2021-09-24,Yes,2022-09-24,0.0


Force Protection:

In [41]:
df = union[union['course_name'].str.contains('Force Protection')]

fp_df = (df.sort_values(['name', 'due_date_1_yr'], ascending=[True, False]).drop_duplicates(['name']).reset_index(drop=True))

fp_df.drop(["due_date_2_yr", "due_date_3_yr"], axis = 1, inplace = True)

fp_df


,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,due_date_1_yr
0,ALEXANDER SARAH,Force Protection (ZZ133079),Maj,SARAH,ALEXANDER,2022-01-03,0.0,2022-01-03,Yes,2023-01-03
1,BARKER DAVID,Force Protection (ZZ133079),Maj,DAVID,BARKER,2021-09-16,0.0,2021-11-09,Yes,2022-11-09
2,BARKER JESSE,Force Protection (ZZ133079),SSgt,JESSE,BARKER,2021-09-02,0.0,2021-09-08,Yes,2022-09-08
3,BAXTER EMELIE,*Force Protection (ZZ133079)-ADLS,TSgt,EMELIE,BAXTER,2021-04-24,NaN,2020-11-30,Yes,2021-11-30
4,BOND JIMMY,Force Protection (ZZ133079),Capt,JIMMY,BOND,2021-09-13,0.0,2021-09-13,Yes,2022-09-13
...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,*Force Protection (ZZ133079)-ADLS,GS-09,DEREK,WILLIAMS,2021-04-24,NaN,2021-01-15,Yes,2022-01-15
71,WILLIAMS GABRIEL,*Force Protection (ZZ133079)-ADLS,GS-12,GABRIEL,WILLIAMS,2021-04-24,NaN,2021-02-22,Yes,2022-02-22
72,WILLIAMSON ASHLEY,Force Protection (ZZ133079),Maj,ASHLEY,WILLIAMSON,2021-10-06,0.0,2021-11-02,Yes,2022-11-02
73,WONG JACQUELINE,Force Protection (ZZ133079),GS-13,JACQUELINE,WONG,2021-09-16,0.0,2022-01-03,Yes,2023-01-03


In [42]:
fp_df['days_overdue'] = today - fp_df['due_date_1_yr']

fp_df['days_overdue'] = round(fp_df['days_overdue'] / pd.Timedelta(1, unit='d'))

fp_df.loc[fp_df['days_overdue'] < 0,'days_overdue'] = 0

fp_df.rename(columns={'due_date_1_yr':'date_due'}, inplace=True)

fp_df.replace({np.nan: 0}, inplace = True)

fp_df['completion_status'] = np.where( ( (fp_df['enrollment_date'] == 0) | (fp_df['days_overdue'] > 0)), "No", "Yes")

fp_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Force Protection (ZZ133079),Maj,SARAH,ALEXANDER,2022-01-03,0.0,2022-01-03,Yes,2023-01-03,0.0
1,BARKER DAVID,Force Protection (ZZ133079),Maj,DAVID,BARKER,2021-09-16,0.0,2021-11-09,Yes,2022-11-09,0.0
2,BARKER JESSE,Force Protection (ZZ133079),SSgt,JESSE,BARKER,2021-09-02,0.0,2021-09-08,Yes,2022-09-08,0.0
3,BAXTER EMELIE,*Force Protection (ZZ133079)-ADLS,TSgt,EMELIE,BAXTER,2021-04-24,0.0,2020-11-30,No,2021-11-30,70.0
4,BOND JIMMY,Force Protection (ZZ133079),Capt,JIMMY,BOND,2021-09-13,0.0,2021-09-13,Yes,2022-09-13,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,*Force Protection (ZZ133079)-ADLS,GS-09,DEREK,WILLIAMS,2021-04-24,0.0,2021-01-15,No,2022-01-15,24.0
71,WILLIAMS GABRIEL,*Force Protection (ZZ133079)-ADLS,GS-12,GABRIEL,WILLIAMS,2021-04-24,0.0,2021-02-22,Yes,2022-02-22,0.0
72,WILLIAMSON ASHLEY,Force Protection (ZZ133079),Maj,ASHLEY,WILLIAMSON,2021-10-06,0.0,2021-11-02,Yes,2022-11-02,0.0
73,WONG JACQUELINE,Force Protection (ZZ133079),GS-13,JACQUELINE,WONG,2021-09-16,0.0,2022-01-03,Yes,2023-01-03,0.0


In [43]:
fp_df = fp_df.assign(course_name = "Force Protection")
fp_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Force Protection,Maj,SARAH,ALEXANDER,2022-01-03,0.0,2022-01-03,Yes,2023-01-03,0.0
1,BARKER DAVID,Force Protection,Maj,DAVID,BARKER,2021-09-16,0.0,2021-11-09,Yes,2022-11-09,0.0
2,BARKER JESSE,Force Protection,SSgt,JESSE,BARKER,2021-09-02,0.0,2021-09-08,Yes,2022-09-08,0.0
3,BAXTER EMELIE,Force Protection,TSgt,EMELIE,BAXTER,2021-04-24,0.0,2020-11-30,No,2021-11-30,70.0
4,BOND JIMMY,Force Protection,Capt,JIMMY,BOND,2021-09-13,0.0,2021-09-13,Yes,2022-09-13,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Force Protection,GS-09,DEREK,WILLIAMS,2021-04-24,0.0,2021-01-15,No,2022-01-15,24.0
71,WILLIAMS GABRIEL,Force Protection,GS-12,GABRIEL,WILLIAMS,2021-04-24,0.0,2021-02-22,Yes,2022-02-22,0.0
72,WILLIAMSON ASHLEY,Force Protection,Maj,ASHLEY,WILLIAMSON,2021-10-06,0.0,2021-11-02,Yes,2022-11-02,0.0
73,WONG JACQUELINE,Force Protection,GS-13,JACQUELINE,WONG,2021-09-16,0.0,2022-01-03,Yes,2023-01-03,0.0


Religious Freedom:

In [44]:
df = union[union['course_name'].str.contains('Religious Freedom')]

rf_df = (df.sort_values(['name', 'due_date_3_yr'], ascending=[True, False]).drop_duplicates(['name']).reset_index(drop=True))

rf_df.drop(["due_date_1_yr", "due_date_2_yr"], axis = 1, inplace = True)

rf_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,due_date_3_yr
0,ALEXANDER SARAH,*Religious Freedom Training (ZZ133109)-ADLS,Maj,SARAH,ALEXANDER,2021-04-28,NaN,2019-06-03,Yes,2022-06-02
1,BARKER DAVID,Religious Freedom Training (ZZ133109),Maj,DAVID,BARKER,2021-09-15,0.0,2021-11-01,Yes,2024-10-31
2,BARKER JESSE,*Religious Freedom Training (ZZ133109)-ADLS,SSgt,JESSE,BARKER,2021-04-27,NaN,2020-01-09,Yes,2023-01-08
3,BAXTER EMELIE,*Religious Freedom Training (ZZ133109)-ADLS,TSgt,EMELIE,BAXTER,2021-04-27,NaN,2021-01-19,Yes,2024-01-19
4,BOND JIMMY,*Religious Freedom Training (ZZ133109)-ADLS,Capt,JIMMY,BOND,2021-04-27,NaN,2019-10-23,Yes,2022-10-22
...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Religious Freedom Training (ZZ133109),GS-09,DEREK,WILLIAMS,2021-09-15,0.0,2021-10-19,Yes,2024-10-18
71,WILLIAMS GABRIEL,*Religious Freedom Training (ZZ133109)-ADLS,GS-12,GABRIEL,WILLIAMS,2021-04-28,NaN,2019-05-28,Yes,2022-05-27
72,WILLIAMSON ASHLEY,Religious Freedom Training (ZZ133109),Maj,ASHLEY,WILLIAMSON,2021-10-06,0.0,2021-11-02,Yes,2024-11-01
73,WONG JACQUELINE,*Religious Freedom Training (ZZ133109)-ADLS,GS-13,JACQUELINE,WONG,2021-04-27,NaN,2020-05-12,Yes,2023-05-12


In [45]:
rf_df['days_overdue'] = today - rf_df['due_date_3_yr']

rf_df['days_overdue'] = round(rf_df['days_overdue'] / pd.Timedelta(1, unit='d'))

rf_df.loc[rf_df['days_overdue'] < 0,'days_overdue'] = 0

rf_df.rename(columns={'due_date_3_yr':'date_due'}, inplace=True)

rf_df.replace({np.nan: 0}, inplace = True)

rf_df['completion_status'] = np.where( ( (rf_df['enrollment_date'] == 0) | (rf_df['days_overdue'] > 0)), "No", "Yes")

rf_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,*Religious Freedom Training (ZZ133109)-ADLS,Maj,SARAH,ALEXANDER,2021-04-28 00:00:00,0.0,2019-06-03 00:00:00,Yes,2022-06-02 00:00:00,0.0
1,BARKER DAVID,Religious Freedom Training (ZZ133109),Maj,DAVID,BARKER,2021-09-15 00:00:00,0.0,2021-11-01 00:00:00,Yes,2024-10-31 00:00:00,0.0
2,BARKER JESSE,*Religious Freedom Training (ZZ133109)-ADLS,SSgt,JESSE,BARKER,2021-04-27 00:00:00,0.0,2020-01-09 00:00:00,Yes,2023-01-08 00:00:00,0.0
3,BAXTER EMELIE,*Religious Freedom Training (ZZ133109)-ADLS,TSgt,EMELIE,BAXTER,2021-04-27 00:00:00,0.0,2021-01-19 00:00:00,Yes,2024-01-19 00:00:00,0.0
4,BOND JIMMY,*Religious Freedom Training (ZZ133109)-ADLS,Capt,JIMMY,BOND,2021-04-27 00:00:00,0.0,2019-10-23 00:00:00,Yes,2022-10-22 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Religious Freedom Training (ZZ133109),GS-09,DEREK,WILLIAMS,2021-09-15 00:00:00,0.0,2021-10-19 00:00:00,Yes,2024-10-18 00:00:00,0.0
71,WILLIAMS GABRIEL,*Religious Freedom Training (ZZ133109)-ADLS,GS-12,GABRIEL,WILLIAMS,2021-04-28 00:00:00,0.0,2019-05-28 00:00:00,Yes,2022-05-27 00:00:00,0.0
72,WILLIAMSON ASHLEY,Religious Freedom Training (ZZ133109),Maj,ASHLEY,WILLIAMSON,2021-10-06 00:00:00,0.0,2021-11-02 00:00:00,Yes,2024-11-01 00:00:00,0.0
73,WONG JACQUELINE,*Religious Freedom Training (ZZ133109)-ADLS,GS-13,JACQUELINE,WONG,2021-04-27 00:00:00,0.0,2020-05-12 00:00:00,Yes,2023-05-12 00:00:00,0.0


In [46]:
rf_df = rf_df.assign(course_name = "Religious Freedom")
rf_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Religious Freedom,Maj,SARAH,ALEXANDER,2021-04-28 00:00:00,0.0,2019-06-03 00:00:00,Yes,2022-06-02 00:00:00,0.0
1,BARKER DAVID,Religious Freedom,Maj,DAVID,BARKER,2021-09-15 00:00:00,0.0,2021-11-01 00:00:00,Yes,2024-10-31 00:00:00,0.0
2,BARKER JESSE,Religious Freedom,SSgt,JESSE,BARKER,2021-04-27 00:00:00,0.0,2020-01-09 00:00:00,Yes,2023-01-08 00:00:00,0.0
3,BAXTER EMELIE,Religious Freedom,TSgt,EMELIE,BAXTER,2021-04-27 00:00:00,0.0,2021-01-19 00:00:00,Yes,2024-01-19 00:00:00,0.0
4,BOND JIMMY,Religious Freedom,Capt,JIMMY,BOND,2021-04-27 00:00:00,0.0,2019-10-23 00:00:00,Yes,2022-10-22 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Religious Freedom,GS-09,DEREK,WILLIAMS,2021-09-15 00:00:00,0.0,2021-10-19 00:00:00,Yes,2024-10-18 00:00:00,0.0
71,WILLIAMS GABRIEL,Religious Freedom,GS-12,GABRIEL,WILLIAMS,2021-04-28 00:00:00,0.0,2019-05-28 00:00:00,Yes,2022-05-27 00:00:00,0.0
72,WILLIAMSON ASHLEY,Religious Freedom,Maj,ASHLEY,WILLIAMSON,2021-10-06 00:00:00,0.0,2021-11-02 00:00:00,Yes,2024-11-01 00:00:00,0.0
73,WONG JACQUELINE,Religious Freedom,GS-13,JACQUELINE,WONG,2021-04-27 00:00:00,0.0,2020-05-12 00:00:00,Yes,2023-05-12 00:00:00,0.0


Records Management:

In [47]:
df = union[union['course_name'].str.contains('User') & (union['course_name'].str.contains('Records'))]

rm_df = (df.sort_values(['name', 'due_date_1_yr'], ascending=[True, False]).drop_duplicates(['name']).reset_index(drop=True))

rm_df.drop(["due_date_2_yr", "due_date_3_yr"], axis = 1, inplace = True)

rm_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,due_date_1_yr
0,ALEXANDER SARAH,User Records Management Training,Maj,SARAH,ALEXANDER,2021-08-02,0.0,2021-08-02,Yes,2022-08-02
1,BARKER DAVID,"AFQTPXXXXX-222RA, Records Management - User Trng",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
2,BARKER JESSE,"AFQTPXXXXX-222RA, Records Management - User Trng",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
3,BAXTER EMELIE,"AFQTPXXXXX_222RA, Records Management - User Tr...",TSgt,EMELIE,BAXTER,2021-05-17,NaN,2020-10-05,Yes,2021-10-05
4,BOND JIMMY,User Records Management Training,Capt,JIMMY,BOND,2021-07-21,0.0,2021-07-21,Yes,2022-07-21
...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,User Records Management Training,GS-09,DEREK,WILLIAMS,2021-07-09,0.0,2021-07-09,Yes,2022-07-09
71,WILLIAMS GABRIEL,User Records Management Training,GS-12,GABRIEL,WILLIAMS,2021-07-20,0.0,2021-07-20,Yes,2022-07-20
72,WILLIAMSON ASHLEY,"AFQTPXXXXX-222RA, Records Management - User Trng",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
73,WONG JACQUELINE,"AFQTPXXXXX-222RA, Records Management - User Trng",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT


In [48]:
rm_df['days_overdue'] = today - rm_df['due_date_1_yr']

rm_df['days_overdue'] = round(rm_df['days_overdue'] / pd.Timedelta(1, unit='d'))

rm_df.loc[rm_df['days_overdue'] < 0,'days_overdue'] = 0

rm_df.rename(columns={'due_date_1_yr':'date_due'}, inplace=True)

rm_df.replace({np.nan: 0}, inplace = True)

rm_df['completion_status'] = np.where( ( (rm_df['enrollment_date'] == 0) | (rm_df['days_overdue'] > 0)), "No", "Yes")

rm_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,User Records Management Training,Maj,SARAH,ALEXANDER,2021-08-02 00:00:00,0.0,2021-08-02 00:00:00,Yes,2022-08-02 00:00:00,0.0
1,BARKER DAVID,"AFQTPXXXXX-222RA, Records Management - User Trng",0,0,0,0,0.0,0,No,0,0.0
2,BARKER JESSE,"AFQTPXXXXX-222RA, Records Management - User Trng",0,0,0,0,0.0,0,No,0,0.0
3,BAXTER EMELIE,"AFQTPXXXXX_222RA, Records Management - User Tr...",TSgt,EMELIE,BAXTER,2021-05-17 00:00:00,0.0,2020-10-05 00:00:00,No,2021-10-05 00:00:00,126.0
4,BOND JIMMY,User Records Management Training,Capt,JIMMY,BOND,2021-07-21 00:00:00,0.0,2021-07-21 00:00:00,Yes,2022-07-21 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,User Records Management Training,GS-09,DEREK,WILLIAMS,2021-07-09 00:00:00,0.0,2021-07-09 00:00:00,Yes,2022-07-09 00:00:00,0.0
71,WILLIAMS GABRIEL,User Records Management Training,GS-12,GABRIEL,WILLIAMS,2021-07-20 00:00:00,0.0,2021-07-20 00:00:00,Yes,2022-07-20 00:00:00,0.0
72,WILLIAMSON ASHLEY,"AFQTPXXXXX-222RA, Records Management - User Trng",0,0,0,0,0.0,0,No,0,0.0
73,WONG JACQUELINE,"AFQTPXXXXX-222RA, Records Management - User Trng",0,0,0,0,0.0,0,No,0,0.0


In [49]:
rm_df = rm_df.assign(course_name = "Records Management")
rm_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Records Management,Maj,SARAH,ALEXANDER,2021-08-02 00:00:00,0.0,2021-08-02 00:00:00,Yes,2022-08-02 00:00:00,0.0
1,BARKER DAVID,Records Management,0,0,0,0,0.0,0,No,0,0.0
2,BARKER JESSE,Records Management,0,0,0,0,0.0,0,No,0,0.0
3,BAXTER EMELIE,Records Management,TSgt,EMELIE,BAXTER,2021-05-17 00:00:00,0.0,2020-10-05 00:00:00,No,2021-10-05 00:00:00,126.0
4,BOND JIMMY,Records Management,Capt,JIMMY,BOND,2021-07-21 00:00:00,0.0,2021-07-21 00:00:00,Yes,2022-07-21 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Records Management,GS-09,DEREK,WILLIAMS,2021-07-09 00:00:00,0.0,2021-07-09 00:00:00,Yes,2022-07-09 00:00:00,0.0
71,WILLIAMS GABRIEL,Records Management,GS-12,GABRIEL,WILLIAMS,2021-07-20 00:00:00,0.0,2021-07-20 00:00:00,Yes,2022-07-20 00:00:00,0.0
72,WILLIAMSON ASHLEY,Records Management,0,0,0,0,0.0,0,No,0,0.0
73,WONG JACQUELINE,Records Management,0,0,0,0,0.0,0,No,0,0.0


SAPR:

In [50]:
df = union[union['course_name'].str.contains('Sexual Assault')]

sapr_df = (df.sort_values(['name', 'due_date_1_yr'], ascending=[True, False]).drop_duplicates(['name']).reset_index(drop=True))

sapr_df.drop(["due_date_2_yr", "due_date_3_yr"], axis = 1, inplace = True)

sapr_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,due_date_1_yr
0,ALEXANDER SARAH,Sexual Assault Prevention and Response (SAPR)...,Maj,SARAH,ALEXANDER,2021-04-28,NaN,2020-10-29,Yes,2021-10-29
1,BARKER DAVID,Sexual Assault Prevention and Response (SAPR)...,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
2,BARKER JESSE,2021 Sexual Assault Prevention and Response An...,SSgt,JESSE,BARKER,2021-12-29,NaN,2021-12-29,Yes,2022-12-29
3,BAXTER EMELIE,Sexual Assault Prevention and Response (SAPR)...,TSgt,EMELIE,BAXTER,2021-04-29,NaN,2020-11-20,Yes,2021-11-20
4,BOND JIMMY,Sexual Assault Prevention and Response (SAPR)...,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Sexual Assault Prevention and Response (SAPR)...,GS-09,DEREK,WILLIAMS,2021-04-29,NaN,2020-10-21,Yes,2021-10-21
71,WILLIAMS GABRIEL,Sexual Assault Prevention and Response (SAPR)...,GS-12,GABRIEL,WILLIAMS,2021-04-29,NaN,2020-10-20,Yes,2021-10-20
72,WILLIAMSON ASHLEY,Sexual Assault Prevention and Response (SAPR)...,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT
73,WONG JACQUELINE,Sexual Assault Prevention and Response (SAPR)...,GS-13,JACQUELINE,WONG,2021-04-29,NaN,2020-10-02,Yes,2021-10-02


In [51]:
sapr_df['days_overdue'] = today - sapr_df['due_date_1_yr']

sapr_df['days_overdue'] = round(sapr_df['days_overdue'] / pd.Timedelta(1, unit='d'))

sapr_df.loc[sapr_df['days_overdue'] < 0,'days_overdue'] = 0

sapr_df.rename(columns={'due_date_1_yr':'date_due'}, inplace=True)

sapr_df.replace({np.nan: 0}, inplace = True)

sapr_df['completion_status'] = np.where( ( (sapr_df['enrollment_date'] == 0) | (sapr_df['days_overdue'] > 0)), "No", "Yes")

sapr_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Sexual Assault Prevention and Response (SAPR)...,Maj,SARAH,ALEXANDER,2021-04-28 00:00:00,0.0,2020-10-29 00:00:00,No,2021-10-29 00:00:00,102.0
1,BARKER DAVID,Sexual Assault Prevention and Response (SAPR)...,0,0,0,0,0.0,0,No,0,0.0
2,BARKER JESSE,2021 Sexual Assault Prevention and Response An...,SSgt,JESSE,BARKER,2021-12-29 00:00:00,0.0,2021-12-29 00:00:00,Yes,2022-12-29 00:00:00,0.0
3,BAXTER EMELIE,Sexual Assault Prevention and Response (SAPR)...,TSgt,EMELIE,BAXTER,2021-04-29 00:00:00,0.0,2020-11-20 00:00:00,No,2021-11-20 00:00:00,80.0
4,BOND JIMMY,Sexual Assault Prevention and Response (SAPR)...,0,0,0,0,0.0,0,No,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,Sexual Assault Prevention and Response (SAPR)...,GS-09,DEREK,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-21 00:00:00,No,2021-10-21 00:00:00,110.0
71,WILLIAMS GABRIEL,Sexual Assault Prevention and Response (SAPR)...,GS-12,GABRIEL,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-20 00:00:00,No,2021-10-20 00:00:00,111.0
72,WILLIAMSON ASHLEY,Sexual Assault Prevention and Response (SAPR)...,0,0,0,0,0.0,0,No,0,0.0
73,WONG JACQUELINE,Sexual Assault Prevention and Response (SAPR)...,GS-13,JACQUELINE,WONG,2021-04-29 00:00:00,0.0,2020-10-02 00:00:00,No,2021-10-02 00:00:00,129.0


In [52]:
sapr_df = sapr_df.assign(course_name = "SAPR")
sapr_df

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,SAPR,Maj,SARAH,ALEXANDER,2021-04-28 00:00:00,0.0,2020-10-29 00:00:00,No,2021-10-29 00:00:00,102.0
1,BARKER DAVID,SAPR,0,0,0,0,0.0,0,No,0,0.0
2,BARKER JESSE,SAPR,SSgt,JESSE,BARKER,2021-12-29 00:00:00,0.0,2021-12-29 00:00:00,Yes,2022-12-29 00:00:00,0.0
3,BAXTER EMELIE,SAPR,TSgt,EMELIE,BAXTER,2021-04-29 00:00:00,0.0,2020-11-20 00:00:00,No,2021-11-20 00:00:00,80.0
4,BOND JIMMY,SAPR,0,0,0,0,0.0,0,No,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,WILLIAMS DEREK,SAPR,GS-09,DEREK,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-21 00:00:00,No,2021-10-21 00:00:00,110.0
71,WILLIAMS GABRIEL,SAPR,GS-12,GABRIEL,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-20 00:00:00,No,2021-10-20 00:00:00,111.0
72,WILLIAMSON ASHLEY,SAPR,0,0,0,0,0.0,0,No,0,0.0
73,WONG JACQUELINE,SAPR,GS-13,JACQUELINE,WONG,2021-04-29 00:00:00,0.0,2020-10-02 00:00:00,No,2021-10-02 00:00:00,129.0


### Step 5: Merging dataframes

Merge all individual course dataframes into one.

In [53]:
frames = [cyber_df, fp_df, rf_df, rm_df, sapr_df]

union2 = pd.concat(frames, ignore_index = 'TRUE')

union2

,name,course_name,rank,first_name,last_name,enrollment_date,grade,date_completed,completion_status,date_due,days_overdue
0,ALEXANDER SARAH,Cyber Awareness,Maj,SARAH,ALEXANDER,2021-03-21 00:00:00,0.0,2021-09-24 00:00:00,Yes,2022-09-24 00:00:00,0.0
1,BARKER DAVID,Cyber Awareness,Maj,DAVID,BARKER,2021-09-16 00:00:00,0.0,2022-01-06 00:00:00,Yes,2023-01-06 00:00:00,0.0
2,BARKER JESSE,Cyber Awareness,SSgt,JESSE,BARKER,2021-03-22 00:00:00,0.0,2021-09-24 00:00:00,Yes,2022-09-24 00:00:00,0.0
3,BAXTER EMELIE,Cyber Awareness,TSgt,EMELIE,BAXTER,2021-10-06 00:00:00,0.0,2021-11-30 00:00:00,Yes,2022-11-30 00:00:00,0.0
4,BOND JIMMY,Cyber Awareness,Capt,JIMMY,BOND,2021-03-22 00:00:00,0.0,2021-09-24 00:00:00,Yes,2022-09-24 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
370,WILLIAMS DEREK,SAPR,GS-09,DEREK,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-21 00:00:00,No,2021-10-21 00:00:00,110.0
371,WILLIAMS GABRIEL,SAPR,GS-12,GABRIEL,WILLIAMS,2021-04-29 00:00:00,0.0,2020-10-20 00:00:00,No,2021-10-20 00:00:00,111.0
372,WILLIAMSON ASHLEY,SAPR,0,0,0,0,0.0,0,No,0,0.0
373,WONG JACQUELINE,SAPR,GS-13,JACQUELINE,WONG,2021-04-29 00:00:00,0.0,2020-10-02 00:00:00,No,2021-10-02 00:00:00,129.0


In [54]:
#Remove unnecessary columns

union2 = union2.drop(["first_name", "last_name", "grade"], axis = 1)

In [56]:
union2 = union2.astype(str)
union2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               375 non-null    object
 1   course_name        375 non-null    object
 2   rank               375 non-null    object
 3   enrollment_date    375 non-null    object
 4   date_completed     375 non-null    object
 5   completion_status  375 non-null    object
 6   date_due           375 non-null    object
 7   days_overdue       375 non-null    object
dtypes: object(8)
memory usage: 23.6+ KB


### Step 6: Creating a .csv

This is what we will use as our data source for Tableau.

In [57]:
outname = 'training_table.csv'

pwd = os.getcwd()

outdir = pwd + "/export_data/"

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

union2.to_csv(fullname)

### We now have our completed results table.

For this project, I'll use Tableau to create a dashboard.